# 广义线性模型 Generalized Linear Models
---
本章主要讲述一些用于回归的方法，其中目标值 y 是输入变量 x 的线性组合。 数学概念表示为：如果$\hat{y}$是预测值，那么有：

$\hat{y}(\theta, x) = \theta^{(0)} + \theta^{(1)} x^{(1)} + ... + \theta^{(p)} x^{(p)}$

在整个模块中，我们定义向量 $w = (\theta_1,..., \theta_p)$ 作为`coef_`(coefficient, 回归系数)，定义 $\theta_0$ 作为 `intercept_`截距.
线性模型虽简单, 却有丰富的变化.考虑单调可微函数$g(\cdot)$
($g(\cdot)$连续且充分光滑), 令

$$y = g^{-1}(\theta^Tx+b)$$
这样得到的模型称为`广义线性模型`,函数$g(\cdot)$称为`联系函数`(link function). 对数线性回归(Logistic Regression)是$g(\cdot)=ln(\cdot)$时的特例

## 线性回归
---

回归的目的是预测数值型的目标值。最直接的办法是依据输入写出一个目标值的计算公式。

假如你想要预测兰博基尼跑车的功率大小，可能会这样计算:

HorsePower = 0.0015 * annualSalary - 0.99 * hoursListeningToPublicRadio

这就是所谓的 `回归方程(regression equation)`，其中的 0.0015 和 -0.99 称作 `回归系数（regression weights）`，求这些回归系数的过程就是回归。

给定数据集 $D = \{(x_1, y_1), (x_2, y_2), \cdots, (x_n, y_n)\}$, 其中$x_i=(x_i^{(1)}, x_i^{(2)}, \cdots, x_i^{(d)}), y_i \in \mathbb{R}$. 线性回归(linear regression)试图学得一个线性模型以尽可能准确地预测实值输出标记.
$$f(x_i) = \theta^T x_i, 使得f(x_i)\simeq y_i$$

对于离散属性, 若属性值之间存在`序`(order)关系, 可通过连续化将其转为连续值; 若属性值间不存在序关系,假定有k个属性值, 则通常将其转为k维向量(OneHotEncoder)

## 普通最小二乘法 Ordinary Least Squares
---
LinearRegression 拟合一个带有系数$w = (\theta^{(0)}, ...,\theta^{(p)})$的线性模型，使得数据集实际观测数据和预测数据（估计值）之间的残差平方和(对应了常用的欧几里得距离Euclidean distance)最小。其数学表达式为:
$$\underset{\theta}{min} {|| X\theta - y||_2}^2$$
求解\theta使$E_{\theta} = \sum_{i=1}^{p}(\theta^Tx_i-y_i)^2$(loss function)最小化的过程, 称为线性回归模型的最小二乘`参数估计`(parameter estimation). $E_{\theta}$是关于$w$的凸函数, 可以令其对$\theta$并令导数为0, 得到$\theta$的最优解的闭式(closed-form)解.

用矩阵形式可以写成$E_{\theta} = (y-X\theta)^T(y-X\theta)$, 对$w$求导得到:
$\frac {\partial E_\theta}{\partial \theta} = 2X^T(X\theta-y)$, 令其为0得到$\hat\theta = (X^TX)^{-1}X^Ty$

需要对矩阵求逆，因此这个方程只在逆矩阵存在的时候适用，我们在程序代码中对此作出判断。 判断矩阵是否可逆的一个可选方案是:

判断矩阵的行列式是否为 0，若为0，矩阵就不存在逆矩阵，不为0的话，矩阵才存在逆矩阵。

[矩阵求导参考1](https://blog.csdn.net/daaikuaichuan/article/details/80620518)

[矩阵求导参考2](http://blog.csdn.net/nomadlx53/article/details/50849941)

例子来自**[机器学习实战](https://github.com/apachecn/AiLearning/blob/master/docs/ml/8.%E5%9B%9E%E5%BD%92.md)**
___
数据格式
## 1. 线性回归
```
x0       x1       y
1.000000	0.067732	3.176513
1.000000	0.427810	3.816464
1.000000	0.995731	4.550095
1.000000	0.738336	4.256571
1.000000	0.981083	4.560815
```

In [ ]:
import numpy as np
from copy import deepcopy
np.set_printoptions(threshold=15)

In [ ]:
class SingularMatrixError(Exception):
    pass

In [ ]:
data = np.loadtxt('data.txt')
data

In [ ]:
# 取X 和 y
X, y = data[:, :-1], data[:, -1]
X  # 保持 X 为 n_sample * n_feature

In [ ]:
class LinearRegression:
    def __init__(self):
        self.intercept_ = None
        self.coef_ = None
    
    @staticmethod
    def least_squares(X, y):
        A = X.T * X
        if np.linalg.det(A) == 0.0:
            print('...')
        w = A.I * X.T * y
        return w
    
    def fit(self, X, y):
        X = np.mat(X)
        y = np.mat(y[:, np.newaxis])
        theta = self.least_squares(X, y)  # n_feature * 1
        self.intercept_ = theta[0, 0]
        self.coef_ = np.array(theta[1:, 0]).ravel()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def regression1():
    data = np.loadtxt('data.txt')
    X, y = data[:, :-1], data[:, -1]
    clf = LinearRegression()
    clf.fit(X, y)
    print(clf.coef_, clf.intercept_)
    
    def line_(x0):
        return clf.coef_[0] * x0 + clf.intercept_
    
    def corr_(X, y):
        # 使用预测值f(x)和y的相关系数来表示, 预测值和实际值的匹配程度
        corr = np.corrcoef(line_(X[:, 1]).ravel(), y)
        print(f'相关系数:\n{corr}\n')
        
    corr_(X, y)
    plt.figure()
    plt.scatter(X[:,1], y[:], c='r', edgecolors='k')
    xmin, xmax = X[:, 1].min(), X[:, 1].max()
    plt.plot([xmin, xmax], [line_(xmin), line_(xmax)])
    plt.show()

In [ ]:
regression1()

In [ ]:
### 

In [ ]:
from sklearn.linear_model import LinearRegression
data = np.loadtxt('data.txt')
X, y = data[:, [1]], data[:, -1]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=123)
clf = LinearRegression()
clf.fit(X, y)
clf.coef_, clf.intercept_

### 普通最小二乘法的复杂度
该方法使用 X 的奇异值分解来计算最小二乘解。如果 X 是一个形状为 (n_samples, n_features)的矩阵，设$$n_{samples} \geq n_{features}$$, 则该方法的复杂度为$$O(n_{samples} n_{fearures}^2)$$

## 2 局部加权线性回归
---
线性回归的一个问题是有可能出现欠拟合现象，因为它求的是具有最小均方差的无偏估计。显而易见，如果模型欠拟合将不能取得最好的预测效果。所以有些方法允许在估计中引入一些偏差，从而降低预测的均方误差。

一个方法是`局部加权线性回归`（Locally Weighted Linear Regression，LWLR）。在这个算法中，我们给预测点附近的每个点赋予一定的权重()，然后与 线性回归 类似，在这个子集上基于最小均方误差来进行普通的回归。我们需要最小化的目标函数大致为:
$$\sum_i w_i(y_i-\theta^Tx_i)^2$$
参考最小二乘法,
$$\begin{aligned}J(\theta)
&= \sum_{i=1}^m w_i(y_i-h_{\theta}(x_i))^2 \\
&= (X\theta - y)^TW(X\theta-y)\end{aligned}$$
$J(\theta)$对$\theta$求导:
$$\nabla_{\theta}J(\theta) = 2X^TWX\theta - 2X^TWy$$
该算法解出回归系数 $\theta$ 的形式如下:
$$\hat \theta = (X^TWX)^{-1}X^TWy$$

LWLR 使用 “核”（与支持向量机中的核类似）来对附近的点赋予更高的权重。核的类型可以自由选择，最常用的核就是高斯核，高斯核对应的权重如下:
$$w(i, i) = exp\left(\frac {(x_i-x)^2}{-2k^2}\right)$$

这样就构建了一个只含对角元素的权重矩阵 w，并且点 x 与 $x_i$ 越近，w(i, i) 将会越大。上述公式中包含一个需要用户指定的参数 k ，它决定了对附近的点赋予多大的权重，这也是使用 LWLR 时唯一需要考虑的参数，下面的图给出了参数 k 与权重的关系。

In [ ]:
def plt_w_(x, k):
    # 预测点x=.5
    x_diff = x - .5
    return np.exp(x_diff.T * x_diff / (-2 * k**2))

In [ ]:
figure = plt.figure()
ax1, ax2, ax3, ax4, ax5 = figure.subplots(5)
ax1.scatter(X[:,0], y[:], c='r', edgecolors='k')
x_ = np.linspace(0, 1, 500)
print(plt_w_(x_, 0.5))
ax2.plot(x_, plt_w_(x_, 0.5), label='k=0.5')
ax2.legend()
ax3.plot(x_, plt_w_(x_, 0.1), label='k=0.1')
ax3.legend()
ax4.plot(x_, plt_w_(x_, 0.01), label='k=0.01')
ax4.legend()
ax5.plot(x_, plt_w_(x_, 10), label='k=10')
ax5.legend()

上面的图是 每个点的权重图（假定我们正预测的点是 x = 0.5），最上面的图是原始数据集，第二个图显示了当 k = 0.5 时，大部分的数据都用于训练回归模型；而最下面的图显示当 k=0.01 时，仅有很少的局部点被用于训练回归模型。

In [ ]:
class LinearRegression_LW:
    def __init__(self, k=1):
        self.k_ = k
        self.X = None
        self.y = None
    
    @staticmethod
    def weight(X, k, test_point):
        # 计算每个测试点时, 其他点的权重
        n_sample = X.shape[0]
        w = np.eye(n_sample)
        for i in range(n_sample):
            diff_X = X[i,:] - test_point
            w[i,i] = np.exp(diff_X * diff_X.T / (-2 * k**2))
        return w
    
    def fit(self, X, y):
        self.X = np.mat(X)
        self.y = np.mat(y[:, np.newaxis])
        
    def predict(self, test_points):
        y_pred = np.zeros(test_points.shape[0]) # n_sample, 
        
        for i in range(test_points.shape[0]):
            test_point = test_points[i, :]
            w = self.weight(self.X, self.k_, test_point)
            A = self.X.T * w * self.X
            if np.linalg.det(A) == 0.0:
                raise SingularMatrixError('奇异矩阵, 用其它方法求')
            theta = A.I * self.X.T * w * self.y
            y_pred[i] = test_point * theta  # 1*nf * n_f*1
        return y_pred
    

In [ ]:
def regression2():
    data = np.loadtxt('data.txt')
    X, y = data[:, :-1], data[:, -1]
    
    fig = plt.figure(figsize=(10, 8))
    for ax, k in zip(fig.subplots(3), (1, 0.01, 0.003)):
        clf = LinearRegression_LW(k)
        clf.fit(X, y)
        ax.scatter(X[:,1], y[:], c='r', edgecolors='k', s=2)
        y_pred = clf.predict(X)
        print(f'k:{k}, errorr: {((y_pred - y)**2).sum()}')
        ind = X[:, 1].argsort()  # x 从小到大排序
        ax.plot(X[:, 1][ind], y_pred[ind])  # 画出 预测的曲线
    plt.show()

In [ ]:
regression2()

上图使用了 3 种不同平滑值绘出的局部加权线性回归的结果。上图中的平滑系数 k =1.0，中图 k = 0.01，下图 k = 0.003 。可以看到，k = 1.0 时的使所有数据等比重，其模型效果与基本的线性回归相同，k=0.01时该模型可以挖出数据的潜在规律，而 k=0.003时则考虑了太多的噪声，进而导致了过拟合现象。

### 示例: 预测鲍鱼的年龄
---
将数据分为测试集和训练集

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
data = np.loadtxt('abalone.txt')
data.shape

In [ ]:
def abalone_age():
    data = np.loadtxt('abalone.txt')
    for i in range(10):
        print("*"*30)
        print(f'第{i+1}次测试:')
        X, y = data[300*i:300*i + 300, :-1], data[300 * i:300*i + 300, -1]  # 每次取300来测试
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=123)
        def error_(y1, y2):
            return ((y1-y2)**2).sum() / len(y1)
        for k in (10, 1, 0.1):  # 不同的核
            clf = LinearRegression_LW(k)
            clf.fit(X_train, y_train)
            try:
                y_pred_train = clf.predict(X_train)
                y_pred_test = clf.predict(X_test)
                print(f"k:{k}, 训练集误差: {error_(y_train, y_pred_train)}, 测试集误差: {error_(y_test, y_pred_test)}")
            except SingularMatrixError as e:
                print(e.args)
                continue
    

In [ ]:
abalone_age()

根据我们上边的测试，可以看出:
简单线性回归达到了与局部加权现行回归类似的效果

## 3 缩减系数来 “理解” 数据
---
如果数据的特征比样本点还多应该怎么办？是否还可以使用线性回归和之前的方法来做预测？答案是否定的，即我们不能再使用前面介绍的方法。这是因为在计算$(X^TX)^{-1}$ 的时候会出错。

如果特征比样本点还多(n_feature > n_sample)，也就是说输入数据的矩阵x可能不是**满秩矩阵**。非满秩矩阵求逆时会出现问题。

为了解决这个问题，我们引入了`岭回归（ridge regression）`这种缩减方法。接着是`lasso法`，最后介绍`前向逐步回归`。

In [ ]:
a = np.arange(12).reshape(3, 4)

In [ ]:
A = np.dot(a.T, a)
np.linalg.det(A)

In [ ]:
b = a.reshape(4, 3)
np.linalg.det(np.dot(b.T, b))

### 3. 1 岭回归
---
岭估计器是普通`LinearRegression`的简单正则化（称为`l2`惩罚）, 通过对系数的大小施加惩罚来解决普通最小二乘法的一些问题.  特别是，它具有的优点是，在计算上不比普通的最小二乘估计更昂贵. 岭回归最小化的是带罚项的残差平方和
$$\underset {\theta}{min} ||X\theta - y||_2^2 + \alpha ||\theta||_2^2$$
其中， $\alpha \geq 0$ 是控制系数收缩量的复杂性参数： $\alpha$ 的值越大，收缩量越大，模型对共线性的鲁棒性也更强。

与上面的操作一样对$\theta$求导, 
$$\nabla_{\theta}J(\theta) = 2X^T(X\theta-y) + 2\alpha I\theta$$
得到最优的$\hat \theta = (X^TX+\alpha I)^{-1}X^Ty$, 它是一个关于$\alpha$的函数.  
可以这样认为: 岭回归就是在矩阵$X^TX$ 上加一个$\alpha I$从而使得矩阵非奇异，进而能对$X^TX+\alpha I$求逆。其中矩阵I是一个 n_feature * n_feature（等于列数)的单位矩阵

岭回归最先用来处理特征数多于样本数的情况，现在也用于在估计中加入偏差，从而得到更好的估计.通过引入惩罚项，能够减少不重要的参数，这个技术在统计学中也叫作 `缩减(shrinkage)`.

In [ ]:
np.logspace(-2, 6, 9)

In [ ]:
def standard(X):
    return (X - X.mean(0)) / X.var(0) 
data = np.loadtxt('abalone.txt')
X, y = data[:, :-1], data[:, -1]

In [ ]:
standard(X)

In [ ]:
def test_alpha():
#     X = np.arange(16).reshape(4, 4)
#     y = np.arange(4).reshape(4, 1)
    data = np.loadtxt('abalone.txt')
    X, y = data[:, :-1], data[:, -1]
    # 需要对数据进行标准化处理
    y = y - y.mean()
    def standard(X):
        return (X - X.mean(0)) / X.var(0) 
    X = standard(X) 
    X = np.mat(X)
    y = np.mat(y[:, np.newaxis])
    I = np.eye(X.shape[1])
    alpha_list = np.logspace(-10, 10, 500)
    theta_list = []
#     print(f"X是奇异矩阵么: {'yes' if not np.linalg.det(X) else 'no'}")
    for alpha in alpha_list:
        A = (np.dot(X.T, X) + alpha * I)
        theta = A.I * X.T * y
        theta_list.append(theta)
    plt.figure(figsize=(10, 8))
    for i in range(X.shape[1]):
        thetas = list(map(lambda x:x[i, 0], theta_list))
        plt.plot(alpha_list, thetas)
    ax = plt.gca()
    # ax.invert_xaxis()
    plt.xscale('log')
    plt.show()
    

In [ ]:
test_alpha()

𝛼 的值越大，收缩量越大，模型对共线性的鲁棒性也更强. 
在最左边，即𝛼 最小时，可以得到所有系数的原始值（与线性回归一致）；而在右边，系数全部缩减为0；在中间部分的某值将可以取得最好的预测效果。为了定量地找到最佳参数值，还需要进行交叉验证。另外，要判断哪些变量对结果预测最具有影响力，在上图中观察它们对应的系数大小就可以了。